In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

In [127]:
data_dir='~/.keras/datasets/twconv_2016_usersec_v9/'
data_dir=os.path.expanduser(data_dir)
fpath = os.path.join(data_dir, 'dialogs.csv')
ds = pd.read_csv(fpath, parse_dates=['timestamp'])
ds.shape

(26181, 16)

In [128]:
data_dir='~/.keras/datasets/twconv_2016_usersec_v9/'
data_dir=os.path.expanduser(data_dir)
fpath = os.path.join(data_dir, 'alldataconvs', 'test_ids.csv')
test_ids_convs = pd.read_csv(fpath)
fpath = os.path.join(data_dir, 'alldatausers', 'test_ids.csv')
test_ids_users = pd.read_csv(fpath)
test_ids_convs.shape, test_ids_users.shape

((1170, 12), (1170, 12))

In [129]:
data_dir='~/data/results/twconvrecsys/'
data_dir=os.path.expanduser(data_dir)
fpath = os.path.join(data_dir, 'twconv_2016_usersec_v9_alldataconvs', 'lstm', 'predictions.csv')
test_preds_convs = pd.read_csv(fpath, header=None)
fpath = os.path.join(data_dir, 'twconv_2016_usersec_v9_alldatausers', 'lstm', 'predictions.csv')
test_preds_users = pd.read_csv(fpath, header=None)
test_preds_convs.shape, test_preds_users.shape

((1170, 10), (1170, 10))

look for the indexes for each popular user

In [130]:
ds.groupby('screen_name').size().sort_values(ascending=False).head(3).index

Index(['MashiRafael', 'aguschmer', 'KarlaMoralesR'], dtype='object', name='screen_name')

In [131]:
from itertools import combinations
domain_users = ['KarlaMoralesR', 'aguschmer', 'MashiRafael']
comb_users = combinations(domain_users, 2)
comb_users = list(comb_users)


In [132]:
users = {}
for name in domain_users:
    conv_ids = ds[ds.screen_name==name].conversation_id
    u = ds[ds.conversation_id.isin(conv_ids)].screen_name.unique()
    users[name] = set(u)

iname = 'intersect'
users[iname] = []

for name1, name2 in comb_users:
    iusers = users[name1].intersection(users[name2])
    users[iname].extend(iusers)
    users[name1] -= iusers
    users[name2] -= iusers
users[iname] = set(users[iname])
    
for name in domain_users:        
    print(len(users[name]))
print(len(users[iname]))

849
1169
2043
181


In [133]:
conversations = {}
for name in domain_users:
    conv_ids = ds[ds.screen_name==name].dialog_id.unique()
    conversations[name] = set(conv_ids)
len(conversations)

conversations[iname] = []

for name1, name2 in comb_users:
    iconvs = conversations[name1].intersection(conversations[name2])
    conversations[iname].extend(iconvs)
    conversations[name1] -= iconvs
    conversations[name2] -= iconvs
conversations[iname] = set(conversations[iname])

for name in domain_users:        
    print(len(conversations[name]))
print(len(conversations[iname]))

953
1404
3298
0


match conversations recommendations in test set

In [139]:
unames=domain_users.copy()
unames.append(iname)
total_preds=0
all_ix = []
data_dir='~/data/results/twconvrecsys/'
data_dir=os.path.expanduser(data_dir)
basepath = os.path.join(data_dir, 'twconv_2016_usersec_v9_alldataconvs_u{}', 'lstm')

for name in unames:
    ix=test_ids_convs[test_ids_convs.source.isin(users[name])].index
    all_ix.extend(list(ix))
    uds = test_preds_convs.iloc[ix,:]
    tds = test_ids_convs.iloc[ix,:]
    bpath = basepath.format(name)
    print(name, uds.shape, bpath)
    os.makedirs(bpath, exist_ok=True)
    fpath = os.path.join(bpath, 'test.csvrecords')
    tds.to_csv(fpath, index=False)    
    fpath = os.path.join(bpath, 'predictions.csv')
    uds.to_csv(fpath, index=False, header=None)
    total_preds += uds.shape[0]
print(total_preds)
print(len(set(all_ix)))

KarlaMoralesR (168, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldataconvs_uKarlaMoralesR/lstm
aguschmer (229, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldataconvs_uaguschmer/lstm
MashiRafael (622, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldataconvs_uMashiRafael/lstm
intersect (154, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldataconvs_uintersect/lstm
1173
1170


In [140]:
total_preds=0
all_ix = []
data_dir='~/data/results/twconvrecsys/'
data_dir=os.path.expanduser(data_dir)
basepath = os.path.join(data_dir, 'twconv_2016_usersec_v9_alldatausers_u{}', 'lstm')

for name in unames:
    ix=test_ids_users[test_ids_users.source.isin(conversations[name])].index
    all_ix.extend(list(ix))
    uds = test_preds_users.iloc[ix,:]
    tds = test_ids_users.iloc[ix,:]
    bpath = basepath.format(name)
    print(name, uds.shape, bpath)
    os.makedirs(bpath, exist_ok=True)
    fpath = os.path.join(bpath, 'test.csvrecords')
    tds.to_csv(fpath, index=False)
    fpath = os.path.join(bpath, 'predictions.csv')
    uds.to_csv(fpath, index=False, header=None)
    total_preds += uds.shape[0]
print(total_preds)
print(len(set(all_ix)))

KarlaMoralesR (228, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldatausers_uKarlaMoralesR/lstm
aguschmer (289, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldatausers_uaguschmer/lstm
MashiRafael (653, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldatausers_uMashiRafael/lstm
intersect (0, 10) /Users/johnnytorres/data/results/twconvrecsys/twconv_2016_usersec_v9_alldatausers_uintersect/lstm
1170
1170
